## SECCIÓN 6

### PARTE 1

Primero comprobaremos las predicciones del modelo para valores altos. Nuestro modelo utilizado para esta sección será el escogido en la parte anterior y que se ha guardado en modelo.pkl.

In [1]:
import pandas as pd
import numpy as np
import joblib  
# Cargamos el modelo final
final_model = joblib.load("modelo_final.pkl")

# Cargamos el dataset
wind_ava = pd.read_csv('wind_ava.csv.gz', compression="gzip")
all_columns = wind_ava.columns.tolist()
sotavento_columns = ['energy'] + [col for col in all_columns if '.13' in col]

wind_ava_sotavento = wind_ava[sotavento_columns]

y_full = wind_ava_sotavento['energy']
X_full = wind_ava_sotavento.drop('energy', axis=1)

y_pred = final_model.predict(X_full)


#Consideramos el tercer cuantil el threshold para valores altos de energía

energy_threshold= wind_ava_sotavento["energy"].quantile(0.75)
print("Threshold de energía",energy_threshold)

# Ahora, creamos dos subsets. 
# Uno de valores considerados bajos y otro con los valores altos de energía

# Filas con valor de energía mayor que el threshold
high_energy_rows=y_full >= energy_threshold

# Filas con valor de energía menor que el threshold
low_energy_rows=y_full<energy_threshold

# Se crean subsets con estas filas
high_energy_subset= X_full[high_energy_rows]
low_energy_subset= X_full[low_energy_rows]

# Predicciones
high_energy_prediction=final_model.predict(high_energy_subset)
low_energy_prediction=final_model.predict(low_energy_subset)

# Cálculo de errores para los dos subsets
high_energy_mae= np.mean(np.abs(high_energy_prediction-y_full[high_energy_rows]))
high_energy_mse= np.mean((high_energy_prediction-y_full[high_energy_rows])**2)


Threshold de energía 1089.375


Ahora imprimimos los resultados y comprobamos si el modelo predice mejor cuando los valores de energía son altos o bajos

In [2]:
# Mostrar resultados
print("MÉTRICAS PARA VALORES DE ALTA ENERGÍA\n ")
print("Error absoluto medio para valores de alta energía\n",high_energy_mae)
print("Error cuadrático medio para valores de alta energía\n",high_energy_mse)


low_energy_mae=np.mean(np.abs(low_energy_prediction)-y_full[low_energy_rows])
low_energy_mse= np.mean((low_energy_prediction-y_full[low_energy_rows])**2)


print("\nMÉTRICAS PARA VALORES DE BAJA ENERGÍA\n ")
print("Error absoluto medio para valores de baja energía\n",low_energy_mae)
print("Error cuadrático medio para valores de baja energía\n",low_energy_mse)

if high_energy_mae>low_energy_mae:
    print("\nEl error absoluto medio es mayor en valores altos de energía\n")

else:
    print("\nEl error absoluto medio es mayor en valores bajos de energía\n")

if high_energy_mse>low_energy_mse:
    print("El error cuadrático medio es mayor en valores altos de energía\n")

else:
    print("El error cuadrático medio es mayor en valores bajos de energía\n")

MÉTRICAS PARA VALORES DE ALTA ENERGÍA
 
Error absoluto medio para valores de alta energía
 4.763981962342188e-05
Error cuadrático medio para valores de alta energía
 1.6382511675874368e-08

MÉTRICAS PARA VALORES DE BAJA ENERGÍA
 
Error absoluto medio para valores de baja energía
 1.138692306768182e-05
Error cuadrático medio para valores de baja energía
 5.225428846854651e-09

El error absoluto medio es mayor en valores altos de energía

El error cuadrático medio es mayor en valores altos de energía



### PARTE 2

En esta parte convertiremos el problema de regresión en uno de clasificación. Para ello, definiremos un threshold de energía(el tercer cuantil), a partir del cual los valores de energía serán considerados altos, y los que sean menores que este threshold serán considerados bajos.

Usaremos algunos de los valores de la parte 1, ya que en ella también hemos usado el threshold del tercer cuantil y hemos creado los subsets de energía alta y baja.

In [3]:

from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline

#Crear una columna de categoría. Los valores con energía menor que el tercer cuantil
#son de la categoría low, y los demás son de la categoría high.
categories=pd.cut(wind_ava_sotavento["energy"],bins=[float('-inf'),energy_threshold,float('inf')],labels=['low','high'])
wind_ava_sotavento=wind_ava_sotavento.copy()
wind_ava_sotavento.loc[:,"category"]= categories
wind_ava_sotavento=wind_ava_sotavento.drop('energy',axis=1)
# Hacemos split del dataset en features y target variable
X = wind_ava_sotavento.drop('category', axis=1)
y = wind_ava_sotavento['category']

# Hacemos split del dataset en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=438770)
outer_cv = KFold(n_splits=5, shuffle=True, random_state=438770)
# Definir scalers
scalers = {
    "StandardScaler": StandardScaler(),
    "MinMaxScaler": MinMaxScaler(),
    "RobustScaler": RobustScaler()
}

# Diccionario para guardar la puntuación de cross validation de cada scaler
outer_scores = {}

# Loop through the scalers
for name, scaler in scalers.items():
    # Create a pipeline with scaler and KNN
    pipeline = Pipeline([
        ('scaler', scaler),
        ('knn', KNeighborsClassifier())
    ])
    
    
    accuracy_scores = cross_val_score(pipeline, X_train, y_train, scoring='accuracy', cv=outer_cv)
    #Obtenemos la media de accuracy de cada scaler
    
    outer_scores[name] = accuracy_scores.mean()  

# Imprimimos los resultados
print("\nResumen de la accuracy para los diferentes scalers con KNN(outer evaluation):")
for scaler, accuracy in outer_scores.items():
    print(f"{scaler}: {accuracy:.4f}")



Resumen de la accuracy para los diferentes scalers con KNN(outer evaluation):
StandardScaler: 0.8476
MinMaxScaler: 0.8462
RobustScaler: 0.8507


Como se puede ver, el robust scaler es el más preciso para esta tarea. Por lo tanto, este será el que usaremos en este problema de clasificación a partir de ahora. 

Lo siguiente que haremos es elegir la mejor técnica de clasificación para intentar encontrar el modelo más apropiado.

In [4]:
from sklearn.model_selection import cross_validate, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression, Lasso
from sklearn.svm import SVC
import numpy as np
import time

# Usaremos TimeSeriesSplit
tscv = TimeSeriesSplit(n_splits=5)

# Lista de pipelines para evaluar
models = {
    'KNN': make_pipeline(RobustScaler(), KNeighborsClassifier()),
    'DecisionTreeClassifier': make_pipeline(RobustScaler(), DecisionTreeClassifier(random_state=438770)),
    'LogisticRegressionNormal': make_pipeline(RobustScaler(), LogisticRegression(max_iter=1000)),
    'SVM': make_pipeline(RobustScaler(), SVC())
}


# Diccionario para guardar los resultados
cv_results = {}

# Evaluamos los modelos
for name, model in models.items():
    start_time = time.time()
    accuracy_scores = cross_validate(model, X, y, cv=tscv, scoring='accuracy', return_train_score=True)
    
    end_time = time.time()
    avg_accuracy = accuracy_scores['test_score'].mean()  
    avg_time = (accuracy_scores['fit_time']).mean()
    total_time = end_time - start_time

    cv_results[name] = {
        'Accuracy': avg_accuracy,
        'Average Fit Time': avg_time,
        'Tiempo Evaluación Total': total_time
    }


In [5]:


def print_results(results):
    for model, metrics in results.items():
        print(f"{model}:")
        print(f"    Tiempo de evaluación total: {metrics['Tiempo Evaluación Total']:.4f} segundos")
        print(f"    Average Fit Time: {metrics['Average Fit Time']:.4f} seconds")
        print(f"    Accuracy media: {metrics['Accuracy']:.4f}\n")

print_results(cv_results)

KNN:
    Tiempo de evaluación total: 0.8456 segundos
    Average Fit Time: 0.0101 seconds
    Accuracy media: 0.8450

DecisionTreeClassifier:
    Tiempo de evaluación total: 0.4277 segundos
    Average Fit Time: 0.0764 seconds
    Accuracy media: 0.8099

LogisticRegressionNormal:
    Tiempo de evaluación total: 0.3784 segundos
    Average Fit Time: 0.0582 seconds
    Accuracy media: 0.8063

SVM:
    Tiempo de evaluación total: 1.4264 segundos
    Average Fit Time: 0.1171 seconds
    Accuracy media: 0.8491



### ANÁLISIS
Se puede ver que SVM tiene la mejor accuracy hasta ahora, aunque el tiempo de entrenamiento es mayor que en otras alternativas como KNN.

## AJUSTE DE HIPERPARÁMETROS

In [6]:
from sklearn.model_selection import  GridSearchCV


# Diccionario para guardar los resultados de cada modelo
results = {

}

# Inner cross-validation será KFold con 5 splits
inner_cv = KFold(n_splits=5, shuffle=True, random_state=438770)


KNN

In [7]:
# Pipeline para KNN
knn_pipeline = Pipeline([
    ('scaler', RobustScaler()),
    ('knn', KNeighborsClassifier())
])

# Grid de hiperparámetros para KNN
knn_params = {
    'knn__n_neighbors': range(1, 31), 
    'knn__weights': ['uniform', 'distance'],  
    'knn__algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  
}

# Algoritmo de GridSearch para encontrar los mejores hiperparámetros
knn_grid_search = GridSearchCV(knn_pipeline, knn_params, cv=inner_cv, scoring='accuracy', verbose=1)

# Iniciamos el timer
start_time = time.time()
knn_grid_search.fit(X, y)  
end_time = time.time()

# Mejores parámetros y accuracy
best_params = knn_grid_search.best_params_
best_accuracy = knn_grid_search.best_score_
training_time_hpo = end_time - start_time

# Imprimir los resultados
print(f"Mejores parámetros para KNN: {best_params}")
print(f"Mejor accuracy para KNN después de HPO: {best_accuracy:.4f}")
print(f"Training time de HPO para KNN: {training_time_hpo:.4f} segundos")


Fitting 5 folds for each of 240 candidates, totalling 1200 fits


Mejores parámetros para KNN: {'knn__algorithm': 'auto', 'knn__n_neighbors': 22, 'knn__weights': 'distance'}
Mejor accuracy para KNN después de HPO: 0.8623
Training time de HPO para KNN: 110.4324 segundos


DecisionTreeClassifier

In [8]:
# Pipeline para el TreeClassifier
tree_pipeline = Pipeline([
    ('scaler', RobustScaler()), 
    ('tree', DecisionTreeClassifier(random_state=438770))
])

# Definición de hiperparámetros
tree_params = {
    'tree__max_depth': range(1, 21), 
    'tree__min_samples_split': range(2, 11), 
}

# GridSearchCV para DecisionTree
tree_grid_search = GridSearchCV(tree_pipeline, tree_params, cv=inner_cv, scoring='accuracy', verbose=1)

# Timing the hyperparameter optimization process
start_time = time.time()
tree_grid_search.fit(X, y)  
end_time = time.time()

# Extracting the results
best_params_tree = tree_grid_search.best_params_
best_accuracy_tree = tree_grid_search.best_score_
training_time_tree_hpo = end_time - start_time

# Output the results for the Regression Tree
print(f"Mejores parámetros para DecisionTreeClassifier: {best_params_tree}")
print(f"Mejor accuracy para DecisionTreeClassifier después de HPO: {best_accuracy_tree:.4f}")
print(f"Tiempo de entrenamiento de HPO para DecisionTreeClassifier: {training_time_tree_hpo:.4f} segundos")


Fitting 5 folds for each of 180 candidates, totalling 900 fits
Mejores parámetros para DecisionTreeClassifier: {'tree__max_depth': 4, 'tree__min_samples_split': 2}
Mejor accuracy para DecisionTreeClassifier después de HPO: 0.8629
Tiempo de entrenamiento de HPO para DecisionTreeClassifier: 90.8962 segundos


LogisticRegression

In [9]:

import time
# Adjust the Lasso Regression pipeline
logistic_regression_pipeline = Pipeline([
    ('scaler', RobustScaler()),
    ('logistic_regression', LogisticRegression(max_iter=10000))  # Initially setting max_iter to 10,000
])

# Grid de hiperparámetros para LogisticRegression
logistic_regression_params = {
    'logistic_regression__penalty': ['l1', 'l2'],                # Regularization penalty
    'logistic_regression__C': [0.001, 0.01, 0.1, 1, 10, 100],    
    'logistic_regression__solver': ['liblinear', 'saga']        # Algoritmo de optimización
}

# GridSearchCV para logistic regression
logistic_regression_grid_search = GridSearchCV(logistic_regression_pipeline, logistic_regression_params, cv=inner_cv, scoring='accuracy', verbose=1,n_jobs=-1)

# Comenzar optimización de hiperparámetros
start_time = time.time()
logistic_regression_grid_search.fit(X, y)  
end_time = time.time()

# Extraemos resultados
best_params_logistic_regression = logistic_regression_grid_search.best_params_
best_accuracy_logistic_regression=logistic_regression_grid_search.best_score_
training_time_logistic_regression_hpo = end_time - start_time

# Imprimir los resultados
print(f"Mejores parámetros para LogisticRegression: {best_params_logistic_regression}")
print(f"Mejor accuracy para LogisticRegression después de HPO: {best_accuracy_logistic_regression:.4f}")
print(f"HPO training time para LogisticRegression: {training_time_logistic_regression_hpo:.4f} segundos")

Fitting 5 folds for each of 24 candidates, totalling 120 fits


Mejores parámetros para LogisticRegression: {'logistic_regression__C': 10, 'logistic_regression__penalty': 'l1', 'logistic_regression__solver': 'liblinear'}
Mejor accuracy para LogisticRegression después de HPO: 0.8193
HPO training time para LogisticRegression: 129.9647 segundos


SVM

In [10]:
# Definimos la técnica de inner cross validation


# Pipeline para SVM
svm_pipeline = Pipeline([
    ('scaler', RobustScaler()),  
    ('svm', SVC())
])

# Define the hyperparameters grid for SVM
svm_params = {
    'svm__C': [0.1,1,10,100], 
    'svm__kernel': ['linear', 'poly', 'rbf'], 
    'svm__gamma': ['scale', 'auto'],  
}

# GridSearchCV para SVM
svm_grid_search = GridSearchCV(svm_pipeline, svm_params, cv=inner_cv, scoring='accuracy', verbose=1,n_jobs=-1)

# Iniciamos el timer para optimización de hiperparámetros
start_time = time.time()
svm_grid_search.fit(X, y) 
end_time = time.time()

# Extraemos los mejores parámetros y el RMSE
best_params_svm = svm_grid_search.best_params_
best_accuracy_svm = svm_grid_search.best_score_
training_time_svm_hpo = end_time - start_time

# Imprimimos los resultados
print(f"Mejores parámetros para SVM: {best_params_svm}")
print(f"Mejor accuracy para SVM después de HPO: {best_accuracy_svm:.4f}")
print(f"Tiempo de HPO para SVM: {training_time_svm_hpo:.4f} segundos")


Fitting 5 folds for each of 24 candidates, totalling 120 fits


Mejores parámetros para SVM: {'svm__C': 100, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
Mejor accuracy para SVM después de HPO: 0.8774
Tiempo de HPO para SVM: 112.1333 segundos


In [11]:
# Storing results for each model in the 'results' dictionary
results['KNN'] = (best_params, best_accuracy, training_time_hpo)
results['ClassificationTree'] = (best_params_tree, best_accuracy_tree, training_time_tree_hpo)
results['SVM'] = (best_params_svm, best_accuracy_svm, training_time_svm_hpo)
results['Logistic regression'] = (best_params_logistic_regression, best_accuracy_logistic_regression, training_time_logistic_regression_hpo)

print("Resultado modelos:\n")
for model_name, (params, accuracy, time) in results.items():
    print(f"{model_name} Resultados:")
    print(f"    Mejores Parámetros: {params}")
    print(f"    Mejor Accuracy: {accuracy:.4f}")
    print(f"    Tiempo entrenamiento HPO: {time:.4f} seconds\n")


Resultado modelos:

KNN Resultados:
    Mejores Parámetros: {'knn__algorithm': 'auto', 'knn__n_neighbors': 22, 'knn__weights': 'distance'}
    Mejor Accuracy: 0.8623
    Tiempo entrenamiento HPO: 110.4324 seconds

ClassificationTree Resultados:
    Mejores Parámetros: {'tree__max_depth': 4, 'tree__min_samples_split': 2}
    Mejor Accuracy: 0.8629
    Tiempo entrenamiento HPO: 90.8962 seconds

SVM Resultados:
    Mejores Parámetros: {'svm__C': 100, 'svm__gamma': 'scale', 'svm__kernel': 'rbf'}
    Mejor Accuracy: 0.8774
    Tiempo entrenamiento HPO: 112.1333 seconds

Logistic regression Resultados:
    Mejores Parámetros: {'logistic_regression__C': 10, 'logistic_regression__penalty': 'l1', 'logistic_regression__solver': 'liblinear'}
    Mejor Accuracy: 0.8193
    Tiempo entrenamiento HPO: 129.9647 seconds



## Análisis

Basándonos en los resultados presentados, el modelo SVM muestra la mayor precisión media después de la optimización de hiperparámetros (0.8657), con un tiempo de entrenamiento de HPO bastante elevado. Sin embargo, al utilizar el parámetro n_jobs=-1(que permite al programa usar más cores de la CPU) este tiempo se ve disminuido. Por lo tanto, el modelo SVM podría considerarse el mejor en términos de precisión para el problema de clasificación en cuestión. 

La LogisticRegression, a pesar de tener un tiempo de entrenamiento de HPO más largo, no proporciona una precisión significativamente mejor que los otros modelos, lo que la convierte en una opción menos atractiva en términos de tiempo de entrenamiento y rendimiento.



Lo siguiente que haremos es comparar los resultados de nuestro mejor modelo (SVM) con los obtenidos por un DummyClassifier:

In [12]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=438770)
class_counts= wind_ava_sotavento['category'].value_counts()
print("Número de elementos de cada clase en nuestro dataset:")
print(class_counts)
# Creamos el dummy regressor
dummy_regressor = DummyClassifier(strategy="most_frequent")

# Entrenamos el dummy regressor
dummy_regressor.fit(X_train, y_train)

# Hacemos predicciones en el test set
y_pred = dummy_regressor.predict(X_test)

# Calculamos accuracy para el dummy regressor
accuracy = accuracy_score(y_test,y_pred)

print(f"Precisión para DummyClassifier: {accuracy:.4f}")


Número de elementos de cada clase en nuestro dataset:
category
low     3561
high    1187
Name: count, dtype: int64
Precisión para DummyClassifier: 0.7674


Como se puede ver, nuestro mejor modelo supera la precisión del DummyClassifier y, de hecho, todos los modelos que hemos probado la superan.

## OUTER EVALUATION

En esta sección veremos cómo se comporta el modelo escogido, SVM, al realizar predicciones sobre el dataset

In [13]:
# Creación de Pipeline de SVM
final_svm_model = Pipeline([
    ('scaler', RobustScaler()),
    ('svm', SVC(C=100, gamma='scale', kernel='rbf'))
])

# Entrenamos con el train set
final_svm_model.fit(X_train, y_train)

# Hacemos predicciones en el test set
y_pred_svm = final_svm_model.predict(X_test)

# Calculamos la precisión de nuestro modelo 
accuracy_svm = accuracy_score(y_test,y_pred_svm)
print(f"Outer Evaluation - Accuracy en el Test Set para SVM: {accuracy_svm:.4f}")

Outer Evaluation - Accuracy en el Test Set para SVM: 0.8958


Se puede ver que la precisión es alta y podemos concluir que el modelo es efectivo a la hora de clasificar elementos.

El modelo escogido estaría definido, por lo tanto, como: SVC(C=100,gamma='scale',kernel='rbf')